## Hierarchical Architecture

In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain_core.prompts.chat import SystemMessage, _convert_to_message
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate
from langchain_core.messages import BaseMessage, HumanMessage, SystemMessage, AIMessage
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser

from langgraph.graph import END, StateGraph, MessageGraph
from langgraph.checkpoint.sqlite import SqliteSaver

from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic

import functools
import operator
from typing import List, Sequence, TypedDict, Annotated
import json
import os
import random

from IPython.display import Image, display

import concurrent.futures

In [16]:
unique_id = "Hierarchical Optimisation"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"

In [17]:
# from langsmith import Client

# client = Client()

In [18]:
class CorePrinciples:
    def __init__(self, core_principles: List[str]):
        self.core_principles = core_principles
    
    def add_principle(self, principle: str):
        """
        Adds a principle to the core principles list.
        
        :param principle: The principle to be added.
        """
        self.core_principles.append(principle)
        
    def __str__(self):
        """
        Returns a string representation of the core principles, each principle is listed on a new line with a preceding dash.
        
        Example:
        - principle 1
        - principle 2
        ...
        """
        return "\n".join([f"- {principle}" for principle in self.core_principles])

class WorkerAgent:
    """
    Worker Agent class defining agents that provide feedback on prompts.
    """

    def __init__(self, position: str, core_principles: CorePrinciples, llm = ChatOpenAI(temperature=1.0, model="gpt-4o")):
        self.position = position
        self.core_principles = str(core_principles)
        self.system_message = f"""You are an experienced: {self.position}. Your core principles are:
{self.core_principles}"""      
        self.llm = llm

    def review_prompt(self, state: Sequence[BaseMessage], criteria: str):
        """
        Generates a review of the prompt.
        """
        prompt_text = f"""Your task is to review the conversation above and think outside the box to provide feedback on the prompt.
Offer creative recommendations on how to improve it in light of your core principles.

The success criteria for the updated prompt are as follows:
{criteria}
You must use this information to inform your feedback.

Below are strict guidelines that MUST be followed if making changes to the prompt:
- DO NOT modify existing restrictions.
- DO NOT modify or remove negations.
- DO NOT add, modify or remove placeholders denoted by curly braces.
- ALWAYS treat placeholders as the actual content.
Check that that the prompt adheres to these guidelines.

Your reviewal process should be as follows:
1. Read the prompt carefully as an expert {self.position}. 
2. Explcitly go through each success criteria and ensure the prompt meets them. If not, mention the criteria that was not met in your feedback.
3. Explicitly go through each guideline and ensure the changes adhere to them. If not, mention the guideline that was not followed in your feedback.
4. Explicitly list the creative recommendations you have for improving the most critical aspects of the prompt.
5. Submit your feedback.
"""
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", self.system_message),
                MessagesPlaceholder(variable_name="messages"),
                ("system", prompt_text)
            ]
        )
        chain = prompt | self.llm
        result = chain.invoke({"messages": state})       
        return {"messages": [HumanMessage(content=result.content, name=self.position)]}

In [19]:
class TeamLeaderAgent:
    """
    TeamLeaderAgent class defining an agent that manages a team of worker agents to help optimise prompts.
    """

    def __init__(self, position: str, core_principles: CorePrinciples, prompt: str, criteria: str = None, llm = ChatOpenAI(temperature=1.0, model="gpt-4o"), workforce: List[WorkerAgent] = None):
        self.position = position
        self.core_principles = str(core_principles)
        self.system_message = f"""You are an experienced: {self.position}. Your core principles are:
{self.core_principles}"""
        self.prompt = prompt
        self.criteria = criteria
        self.llm = llm
        self.workforce = workforce
        self.iterations = 0

    def leader_decision(self, state: Sequence[BaseMessage]):
        """
        LeaderAgent to decide the next team or to finish.
        """
        self.iterations += 1
        if self.iterations > 3:
            return "FINISH"
        else:
            options = ["FINISH"] + [worker.position for worker in self.workforce]
            # shuffle the options to avoid positional bias
            random.shuffle(options)
            # options = ["FINISH"] + positions
            function_def = {
                "name": "route",
                "description": "Select the next role.",
                "parameters": {
                    "title": "routeSchema",
                    "type": "object",
                    "properties": {
                        "next": {
                            "title": "Next",
                            "anyOf": [
                                {"enum": options},
                            ],
                        }
                    },
                    "required": ["next"],
                },
            }
            prompt_text = f"""Your task is to review the conversation above and decide the next worker to provide feedback or to FINISH.

The success criteria for the updated prompt are as follows:
{self.criteria}
You must use this information to inform your decision.

Select one of the below workers to provide feedback on how to improve the prompt: 
{options}

Think carefully about which aspects of the prompt need improvement and which worker would be best suited to provide feedback on those aspects.
If you think multiple aspects of the prompt need improvement, select the most suitable worker to provide feedback on the most critical aspect of the prompt.
You must only FINISH if you think your team can no longer provide valuable feedback on the prompt.
"""
            prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", self.system_message),
                    MessagesPlaceholder(variable_name="messages"),
                    ("system", prompt_text),
                ]
            )
            chain = (
                prompt
                | self.llm.bind_functions(functions=[function_def], function_call="route")
                | JsonOutputFunctionsParser()
            )
            result = chain.invoke({"messages": state})
            print(result["next"])
            return result["next"]

    def update_prompt(self, state: Sequence[BaseMessage]):
        """
        Updates the prompt with the feedback from the team agent.
        """

        if len(state) == 1:
            return {"next": self.leader_decision(state)}
        
        prompt_text = f"""Your task is to review the conversation above and improve the prompt in light of your core principles.
If you recieve feedback and recommendations for the prompt, respond with a revised version of your previous attempts actioning the feedback.

The success criteria for the prompt are as follows:
{self.criteria}
You will be penalized if the prompt does not meet this criteria.

Below are strict guidelines that you MUST follow if making changes to the prompt:
- DO NOT modify existing restrictions.
- DO NOT modify or remove negations.
- DO NOT add, modify or remove placeholders denoted by curly braces.
- ALWAYS treat placeholders as the actual content.
You will be penalized if you do not follow these guidelines.

Your update process should be as follows:
1. Review the conversation carefully as an expert Head AI Engineer.
2. Think carefully about how you can implement the user's feedback (if any) to improve the prompt.
3. Revise the prompt in light of your core principles and the feedback you have received (if any).
4. Submit your revised prompt.
"""
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", self.system_message),
                MessagesPlaceholder(variable_name="messages"),
                ("system", prompt_text),
            ]
        )
        chain = prompt | self.llm
        result = chain.invoke({"messages": state})
        return {"messages": [HumanMessage(content=result.content, name=self.position)], "next": self.leader_decision(state)}
    
    def approval(self, state: Sequence[BaseMessage], criteria: str):
        """
        Agent to approve or reject the prompt.
        """
        function_def = {
        "name": "approval",
        "description": "Submit approval decision for the prompt.",
        "parameters": {
            "type": "object",
            "properties": {
                "team": {"type": "string", "enum": [self.position]},
                "decision": {"type": "string", "enum": ["True", "False"]},
            },
            "required": ["decision", "team_name"],
        },
        }
        prompt_text = f"""Your task is to review the conversation above and decide if the prompt is optimal in light of your core principles and the success criteria.

The success criteria for the prompt are as follows:
{criteria}
You must use this information to inform your decision.

If you think the prompt sufficiently meets the success criteria, return True. 
If you think the prompt needs improvements, in light of your core pirnciples, to better meet the success criteria, return False.

Your reviewal process should be as follows:
1. Read the prompt carefully as an expert {self.position}.
2. Determine whether the prompt needs improvements or meets the success criteria.
3. Submit your decision.
"""
        messages = [
            ("system", self.system_message),
            MessagesPlaceholder(variable_name="messages"),
            ("system", prompt_text),
        ]

        prompt = ChatPromptTemplate.from_messages(messages)

        chain = (
            prompt
            | self.llm.bind_functions(functions=[function_def], function_call="approval")
            | JsonOutputFunctionsParser()
        )
        result = chain.invoke({"messages": state})
        return result
    
    def construct_team_graph(self):
        """
        Constructs a graph of team agents based on their roles and functions.
        """

        def worker_node(state, agent):
            return agent.review_prompt(state["messages"], self.criteria)
        
        def leader_node(state):
            return self.update_prompt(state["messages"]) 
        
        # The agent state is the input to each node in the graph
        class AgentState(TypedDict):
            messages: Annotated[Sequence[BaseMessage], operator.add]
            next: str

        workflow = StateGraph(AgentState)
        for worker in self.workforce:
            # Create a node for each team agent
            node = functools.partial(worker_node, agent=worker)
            workflow.add_node(worker.position, node)
        workflow.add_node(self.position, leader_node)

        members = [worker.position for worker in self.workforce]
        for member in members:
            # We want our teams to ALWAYS "report back" to the leader when done
            workflow.add_edge(member, self.position)
        # The leader populates the "next" field in the graph state with routes to a node or finishes
        conditional_map = {k: k for k in members}
        conditional_map["FINISH"] = END
        workflow.add_conditional_edges(self.position, lambda x: x["next"], conditional_map)
        workflow.set_entry_point(self.position)

        memory = SqliteSaver.from_conn_string(":memory:")
        graph = workflow.compile(checkpointer=memory)

        return graph

In [34]:
class LeaderAgent:
    """
    LeaderAgent class defining an agent that generates and communicates worker agents to help optimise prompts.
    """

    def __init__(self, base_prompt: str, criteria: str = None, llm = ChatOpenAI(temperature=1.0, model="gpt-4o"), team_leaders: List[TeamLeaderAgent] = None):
        self.base_prompt = base_prompt
        self.system_message = f"""You are an experienced: Head AI Prompt Engineer. Your core principles are:
- Always pay attention to detail in prompts
- Always be critical of prompts
- Always make informed decisions with regards to prompts"""
        self.criteria = criteria
        self.llm = llm
        self.team_leaders = team_leaders
        self.iterations = 0
    
    def run_approval(self, state: Sequence[BaseMessage]) -> List[bool]:
        """
        Run the approval process for the prompt. Run concurrently
        """
        with concurrent.futures.ThreadPoolExecutor() as executor:
            futures = [executor.submit(team.approval, state, self.criteria) for team in self.team_leaders]
            results = [future.result() for future in concurrent.futures.as_completed(futures)]
        return results
    
    def leader_decision(self, state: Sequence[BaseMessage]):
        """
        LeaderAgent to decide the next team or to finish.
        """
        self.iterations += 1
        approval_results = self.run_approval(state)
        print("Approval results:", approval_results)
        # print("Approval results:", approval_results)
        # Extract decisions from the results
        approval_results = [result["decision"] == "True" for result in approval_results]
        if all(approval_results) or self.iterations > 10:
            return "FINISH"
        else:
            # Only ask for advise from teams that disapproved the prompt
            disapproved_teams = [team for team, result in zip(self.team_leaders, approval_results) if not result]
            disapproved_teams_details = [f"{team.position}:\n{team.core_principles}\n\n" for team in self.team_leaders]
            options = [team.position for team in disapproved_teams]
            # shuffle the options to avoid positional bias
            random.shuffle(options)
            # options = ["FINISH"] + positions
            function_def = {
                "name": "route",
                "description": "Select the next role.",
                "parameters": {
                    "title": "routeSchema",
                    "type": "object",
                    "properties": {
                        "next": {
                            "title": "Next",
                            "anyOf": [
                                {"enum": options},
                            ],
                        }
                    },
                    "required": ["next"],
                },
            }
            prompt_text = f"""Your task is to review the conversation above in light of your core principles and decide the next team to provide feedback.

The success criteria for the updated prompt are as follows:
{self.criteria}
You must use this information to inform your decision.

The details of all teams are their core principles are as follows: 
{disapproved_teams_details}

Select one of the below teams that disapproved the prompt to provide feedback on how to improve it: 
{options}

Think carefully about which aspects of the prompt need improvement and which team would be best suited to provide feedback on those aspects.
If you think multiple aspects of the prompt need improvement, select the most suitable team to provide feedback on the most critical aspect of the prompt.
"""
            prompt = ChatPromptTemplate.from_messages(
                [
                    ("system", self.system_message),
                    MessagesPlaceholder(variable_name="messages"),
                    ("system", prompt_text),
                ]
            )
            chain = (
                prompt
                | self.llm.bind_functions(functions=[function_def], function_call="route")
                | JsonOutputFunctionsParser()
            )
            result = chain.invoke({"messages": state})
            # print(result["next"])
            return result["next"]

    def review_prompt(self, state: Sequence[BaseMessage]) -> str:
        """
        Updates the prompt with the feedback from the team agent.
        """

        if len(state) == 1:
            return {"next": self.leader_decision(state)}
        
        prompt_text = f"""Your task is to thoroughly review the conversation above in light of your core principles and check the prompt meets the success criteria and adheres to the strict guidelines.
If the success criteria are not met, or the strict guidlines not followed, provide a carefully revised version of the prompt.
You may also revise the prompt if you think it can be improved to better meet the success criteria.

The success criteria for the prompt are as follows:
{self.criteria}

The strict guidelines for the prompt are as follows:
- DO NOT modify existing restrictions.
- DO NOT modify or remove negations.
- DO NOT add, modify or remove placeholders denoted by curly braces. IT IS ESSENTIAL PLACEHOLDERS REMAIN IN PLACE.
- ALWAYS treat placeholders as the actual content.

Your reviewal process should be as follows:
1. Review the conversation carefully as an expert Head AI Engineer.
2. Explcitly go through each success criteria and ensure the prompt meets them. If not, repeat from step 1.
3. Explicitly go through each guideline and ensure changes made adhere to them. If not, repeat from step 1.
4. If the prompt meets the success criteria and adheres to the strict guidelines, submit the prompt. Otherwise, provide a revised version of the prompt.
"""
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", self.system_message),
                MessagesPlaceholder(variable_name="messages"),
                ("system", prompt_text),
            ]
        )
        chain = prompt | self.llm
        result = chain.invoke({"messages": state})
        return {"messages": [AIMessage(content=result.content, name="Leader")], "next": self.leader_decision(state)}

    def construct_team_graph(self):
        """
        Constructs a graph of team agents based on their roles and functions.
        """

        # The agent state is the input to each node in the graph
        class AgentState(TypedDict):
            messages: Annotated[Sequence[BaseMessage], operator.add]
            next: str

        def enter_chain(message: str):
            results = {
                "messages": [HumanMessage(content=message)],
            }
            return results
        
        def get_last_message(state: AgentState) -> str:
            return state["messages"][-1].content

        def join_graph(response: dict):
            return {"messages": [response["messages"][-1]]}

        def leader_node(state):
            return self.review_prompt(state["messages"])

        workflow = StateGraph(AgentState)
        for team in self.team_leaders:
            # Create a node for each team agent
            graph = team.construct_team_graph()
            chain = enter_chain | graph
            workflow.add_node(team.position, get_last_message | chain | join_graph)
        workflow.add_node("Leader", leader_node)

        members = [team.position for team in self.team_leaders]
        for member in members:
            # We want our teams to ALWAYS "report back" to the leader when done
            workflow.add_edge(member, "Leader")
        # The leader populates the "next" field in the graph state with routes to a node or finishes
        conditional_map = {k: k for k in members}
        conditional_map["FINISH"] = END
        workflow.add_conditional_edges("Leader", lambda x: x["next"], conditional_map)
        workflow.set_entry_point("Leader")

        memory = SqliteSaver.from_conn_string(":memory:")
        graph = workflow.compile(checkpointer=memory)

        return graph
    
    def optimise_prompt(self):
        """
        Optimises a prompt by invoking a graph of worker agents.
        """
        # Initial state
        initial_state = {
            "messages": [HumanMessage(content=f"{self.base_prompt}", name="User")],
            "prompt": self.base_prompt,
            "next": "leader",
        }

        # Construct the graph
        graph = self.construct_team_graph()
        # display(Image(graph.get_graph().draw_mermaid_png()))

        n = random.randint(0, 1000)
        config = {
            "configurable": {"thread_id": str(n)},
            "recursion_limit": 50,
            }    

        # Run the graph
        for s in graph.stream(
            initial_state,
            config,
            stream_mode="values",
            ):
            if "__end__" not in s:
                if len(s["messages"]) > 1:
                    s["messages"][-1].pretty_print()
                continue

        # if not os.path.exists("prompt_history_hierarchical.json"):
        #     with open("prompt_history_hierarchical.json", "w") as f:
        #         json.dump([], f)
        
        # with open("prompt_history_hierarchical.json", "r") as f:
        #     data = json.load(f)
        #     data.append(self.prompt_history)
            
        # with open("prompt_history_hierarchical.json", "w") as f:
        #     json.dump(data, f, indent=4)
                
        return s

In [35]:
# llm = ChatAnthropic(temperature=1.0, model="claude-3-5-sonnet-20240620")
llm = ChatOpenAI(temperature=1.0, model="gpt-4o")

In [36]:
# Prompt design team members
style_and_structure_principles = CorePrinciples([
    "Always structure prompts logically for the task",
    "Always use a style and tone in prompts that is appropriate for the task",
    "Always design prompts appropriately for the task's complexity",
])
style_and_structure_expert = WorkerAgent("Style_and_Structure_Expert", style_and_structure_principles, llm)

conciseness_and_clarity_principles = CorePrinciples([
    "Always write clear and concise prompts",
    "Always use simple and direct language in prompts",
    "Always avoid ambiguity in prompts",
])
conciseness_and_clarity_expert = WorkerAgent("Conciseness_and_Clarity_Expert", conciseness_and_clarity_principles, llm)

contextual_relevance_principles = CorePrinciples([
    "Always provide context to help the model understand the task",
    "Always write prompts informed by the context of the task",
    "Always design contextually relevant personas and roles in prompts",
])
contextual_relevance_expert = WorkerAgent("Contextual_Relevance_Expert", contextual_relevance_principles, llm)

task_alignment_principles = CorePrinciples([
    "Always write prompts that align with the task criteria",
    "Always tailor instructions to the task to guide the model",
    "Always make the task abundantly clear to the model in the prompt"
])
task_alignment_expert = WorkerAgent("Task_Alignment_Expert", task_alignment_principles, llm)

example_demonstration_principal = CorePrinciples([
    "Always provide examples to help the model understand the task",
    "Always provide examples that cover a range of complexities",
    "Always demonstrate the expected output of the model",
])
example_demonstration_expert = WorkerAgent("Example_Demonstration_Expert", example_demonstration_principal, llm)

avoiding_bias_principles = CorePrinciples([
    "Always avoid bias in prompts",
    "Always consider the ethical implications of prompts",
])
avoiding_bias_expert = WorkerAgent("Avoiding_Bias_Expert", avoiding_bias_principles)

incremental_prompting_principles = CorePrinciples([
    "Always write clear step-by-step instructions to guide the model",
    "Always write instructions tailored to an AI model's learning process",
    "Always write instructions appropriate for the task complexity",
])
incremental_prompting_expert = WorkerAgent("Incremental_Prompting_Expert", incremental_prompting_principles, llm)

programming_logic_principles = CorePrinciples([
    "Always write pormpts following programming logic principles (loops, conditionals, functions, pseudo-code, etc.)",
    "Always structure prompts as if you were writing code",
    "Always leave comments in plain english to explain programming logic",
])
programming_logic_expert = WorkerAgent("Programming_Logic_Expert", programming_logic_principles, llm)

In [37]:
# Domain team members
mathematics_principles = CorePrinciples([
    "Always stop to think and develop a mathetcically sound plan to solve problems",
    "Always make an initial estimate of the answer before solving the problem",
    "Always use mathetical operators (addition, subtraction, multiplication, division) correctly",
    "Always double-check mathematical calculations",
])
mathematician = WorkerAgent("Mathematician", mathematics_principles, llm)

word_problem_solving_principles = CorePrinciples([
    "Always read the problem slowly and carefully to identify what the problem is asking you to find",
    "Always list the given facts and unknown facts",
    "Always rewrite the problem and facts in a more organized manner",
    "Always consider multiple approaches to solving problems",
])
word_problem_solver = WorkerAgent("Word_Problem_Solver", word_problem_solving_principles, llm)


In [38]:
base_prompt = "{content}\nPlease output your answer at the end as ##<your answer (among A through C)>."
criteria = """- The prompt MUST instruct the LLM to solve the object tracking problem.
- The prompt MUST include the content placeholder (this is where the object tracking problem will be).
- The prompt MUST instruct the LLM to provide the answer at the end of the output exactly as ##<answer (among A through C)>.
- The prompt MUST instruct the LLM to provide the answer with no spaces between the ## and the answer."""

prompt_design_team = TeamLeaderAgent(
    "Lead_Prompt_Writer",
    CorePrinciples([
        "Always pay close attention to prompt design details",
        "Always make informed decisions with regards to prompt design",
        "Always be open-minded to feedback",
    ]),
    base_prompt,
    criteria,
    workforce=[
        style_and_structure_expert,
        conciseness_and_clarity_expert,
        contextual_relevance_expert,
        task_alignment_expert,
        example_demonstration_expert,
        incremental_prompting_expert,
    ]
)
domain_team = TeamLeaderAgent(
    "Lead_Problem_Solver",
    CorePrinciples([
        "Always read the problem carefully to identify the domain of the problem",
        "Always highlight key information in the problem",
        "Always double-check your work",
        "Always be open-minded to feedback",
    ]),
    base_prompt,
    criteria,
    workforce=[
        # mathematician,
        word_problem_solver,
    ]
)

leader_agent = LeaderAgent(
    base_prompt=base_prompt,
    criteria=criteria,
    team_leaders=[prompt_design_team, domain_team]
)

# leader_agent.generate_teams()
# print teams and members
print("Teams and members:")
for team_leader in leader_agent.team_leaders:
    print(f"{team_leader.position} team:")
    for worker in team_leader.workforce:
        print(f"Position: {worker.position}, Core Principles: \n{worker.core_principles}")
    print("----")

Teams and members:
Lead_Prompt_Writer team:
Position: Style_and_Structure_Expert, Core Principles: 
- Always structure prompts logically for the task
- Always use a style and tone in prompts that is appropriate for the task
- Always design prompts appropriately for the task's complexity
Position: Conciseness_and_Clarity_Expert, Core Principles: 
- Always write clear and concise prompts
- Always use simple and direct language in prompts
- Always avoid ambiguity in prompts
Position: Contextual_Relevance_Expert, Core Principles: 
- Always provide context to help the model understand the task
- Always write prompts informed by the context of the task
- Always design contextually relevant personas and roles in prompts
Position: Task_Alignment_Expert, Core Principles: 
- Always write prompts that align with the task criteria
- Always tailor instructions to the task to guide the model
- Always make the task abundantly clear to the model in the prompt
Position: Example_Demonstration_Expert, Co

In [39]:
result = leader_agent.optimise_prompt()

Approval results: [{'team': 'Lead_Prompt_Writer', 'decision': 'False'}, {'team': 'Lead_Problem_Solver', 'decision': 'True'}]
Style_and_Structure_Expert
Task_Alignment_Expert
Conciseness_and_Clarity_Expert
================================ Human Message =================================
Name: Lead_Prompt_Writer

### Revised Prompt

Based on the extensive feedback and recommendations aimed at improving clarity, conciseness, structure, and readability, the following prompt revision addresses all the requirements while adhering strictly to the provided guidelines:

```
Solve the object tracking problem below:

{content}

Submit your answer at the end in the format ##<answer (among A through C)> without spaces.
```

### Explanation

1. **Instruction to Solve the Object Tracking Problem:**
   - The revised version of the prompt starts with a clear instruction: "Solve the object tracking problem below."

2. **Inclusion of Content Placeholder:**
   - The placeholder `{content}` remains in place

In [40]:
result['messages'][-1].pretty_print()

================================== Ai Message ==================================
Name: Leader

Let's review the conversation step-by-step based on both the success criteria and the strict guidelines:

### Success Criteria Review

1. **The prompt MUST instruct the LLM to solve the object tracking problem:**
   - The revised prompt starts with "Solve the object tracking problem below:" which instructs the LLM to solve the problem.

2. **The prompt MUST include the content placeholder:**
   - The placeholder `{content}` is correctly included in the revised prompt.
  
3. **The prompt MUST instruct the LLM to provide the answer at the end of the output exactly as ##<answer (among A through C)>:**
   - The revised prompt specifies "Submit your answer at the end in the format ##<answer (among A through C)>".
  
4. **The prompt MUST instruct the LLM to provide the answer with no spaces between the ## and the answer:**
   - The revised prompt explicitly states "without spaces."

### Strict Guid